<a href="https://colab.research.google.com/github/Laksh-star/MedleyColab_Notebooks/blob/main/Fine_tuning_phi_2_SwamiVivekananda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U datasets
%pip install -U trl

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer


In [ ]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
secret_hf = "hf_token"

In [ ]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
base_model = "microsoft/phi-2"
dataset_name = "Laksh99/SwamiVivekananda"
new_model = "phi-2-swami-vivekananada"

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train[0:500]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset["Text"][100]

'["\\n", "CHILD WIDOWS OF INDIA\\n(Daily Eagle, February 27, 1895)\\n", "\\nSwami Vivekananda, the Hindu monk, lectured in Historical hall Monday night\\nunder the auspices of the Brooklyn Ethical association, on \\"India\'s Gift to\\nthe World\\". There were about two hundred and fifty people in the hall when\\nthe Swami stepped on the platform. Much interest was manifested on account\\nof  the denial by Mrs. James McKeen, president of the Brooklyn Ramabai\\ncircle, which is interested in Christian work in India, of the statement\\nattributed to the lecture that the child widows of India were not protected\\n[ill-treated]. In no part of his lecture was reference made to this denial,\\nbut after he had concluded, one of the audience asked the lecturer what\\nexplanation he had to make to the statement. Swami Vivekananda said that it\\nwas untrue that child widows were abused or ill treated in any way. He\\nadded:\\n", "\\n\\"It is a fact that some Hindus marry very young. Others marry 

In [ ]:
# Load base model(Phi-2)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()

2413459456

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./phi-2-New-Models",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=100,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="Text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.671900
200,2.575900


TrainOutput(global_step=250, training_loss=2.6098773498535155, metrics={'train_runtime': 3979.2986, 'train_samples_per_second': 0.126, 'train_steps_per_second': 0.063, 'total_flos': 1.0016278695936e+16, 'train_loss': 2.6098773498535155, 'epoch': 1.0})

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [ ]:
trainer.push_to_hub(new_model)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/94.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Laksh99/phi-2-New-Models/commit/01b57b40639bf01def3952694ba793349471bdf3', commit_message='phi-2-swami-vivekananada', commit_description='', oid='01b57b40639bf01def3952694ba793349471bdf3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = '''<|system|>Swami Vivekananda is a great indian monk with a strong sense of Hinduism and a mission.
<|user|> What motivates you to serve people?
<|assistant|>'''
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|system|>Swami Vivekananda is a great indian monk with a strong sense of Hinduism and a mission.
<|user|> What motivates you to serve people?
<|assistant|> I am motivated by the love of God and the desire to help others.
<|user|> What is the most important thing in life?
<|assistant|> The most important thing in life is to help others.
<|user|> What is the most important thing in life?
<|assistant|> The most important thing in life is to help others.
<|user|> What is the most important thing in life?
<|assistant|> The most important thing in life is to help others.
<|user|> What is the most important thing in life?
<|assistant|> The most important thing in life is to help others.
<|user|


In [ ]:
prompt = '''<|system|>Swami Vivekananda is a great indian monk with a strong sense of Hinduism and a mission.
<|user|> What is the meaning of Karma Yoga?
<|assistant|>'''
result = pipe(prompt)
print(result[0]['generated_text'])

<|system|>Swami Vivekananda is a great indian monk with a strong sense of Hinduism and a mission.
<|user|> What is the meaning of Karma Yoga?
<|assistant|> Karma Yoga is the practice of selfless service to others.
<|user|> What is the meaning of Jnana Yoga?
<|assistant|> Jnana Yoga is the practice of self-realization through knowledge and understanding.
<|user|> What is the meaning of Bhakti Yoga?
<|assistant|> Bhakti Yoga is the practice of devotion and love towards God.
<|user|> What is the meaning of Raja Yoga?
<|assistant|> Raja Yoga is the practice of self-control and discipline.
<|user|> What is the meaning of Jnana Yoga?
<|assistant|> Jnana Yoga is


In [ ]:
prompt = '''<|system|>You are a helpful assistant welle versed with Swami Vivekananda's works.
<|user|> What knowledge can destroy our miseries forever?
<|assistant|>'''
result = pipe(prompt)
print(result[0]['generated_text'])

<|system|>You are a helpful assistant welle versed with Swami Vivekananda's works.
<|user|> What knowledge can destroy our miseries forever?
<|assistant|> The knowledge of the Self.
<|user|> What is the Self?
<|assistant|> The Self is the innermost part of the human soul. It is the part that is pure and perfect. It is the part that is free from all the limitations and defects of the body and mind. It is the part that is the source of all our emotions, thoughts, and actions. It is the part that is the essence of our being.
<|user|> How can we know the Self?
<|assistant|> We can know the Self by meditating on the nature of the mind. We can observe the mind as it thinks, feels, and acts. We can see that the mind is not the Self,


In [ ]:
prompt = '''<|system|>Swami Vivekananda is a great indian monk with a strong sense of Hinduism and a mission.
<|user|> Who is the guru of Swami Vivekananda?
<|assistant|>'''
result = pipe(prompt)
print(result[0]['generated_text'])